# BDA2 - Spark SQL

Redo the exercises BDA1 using Spark SQL whenever possible. There are two ways to write queries in SparkSQL - using built-in API functions or running SQL-like
queries. To pass this lab, you need to use built-in API functions for all the questions, in addition, SQL-like queries for the second question. The slides of this link (https://www.ida.liu.se/~732A54/lab/SparkSQLQuickIntro.pdf) show some examples of Spark SQL.

## Setup

The following code is used to prepare the data files, execute only once and **be careful!**

In [1]:
import pyspark
import pyspark.sql
from pyspark.sql import SQLContext, Row
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import col
import csv
import math
import sys
import pandas as pd

tr_path = "../data/temperature-readings-tiny.csv"
tr_big_path = "../data/temperature-readings-big.csv"
pr_path = "../data/precipitation-readings-tiny.csv"
sro_path = "../data/stations-Ostergotland.csv"
sr_path = "../data/stations.csv"

In [2]:
sc = pyspark.SparkContext(appName = "Temperature")
sc_sql = pyspark.SQLContext(sc)

temperature_readings = sc.textFile(tr_path)
temperature_readings_big = sc.textFile(tr_big_path)
precipitation_readings = sc.textFile(pr_path)
station_oster_readings = sc.textFile(sro_path)
station_readings = sc.textFile(sr_path)

## Assignment 1

What are the lowest and highest temperatures measured each year for the period 1950- 2014. Provide the lists sorted in the descending order with respect to the maximum temperature. In this exercise you will use the `temperature-readings.csv` file.

In [11]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(year=int(p[1].split("-")[0]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014)).groupBy('year')
res = res.agg(F.max('temp').alias('temp_max'), F.min('temp').alias('temp_min'))

global df

df = res.orderBy('temp_max', ascending=False).toPandas()

CPU times: user 52.1 ms, sys: 12.1 ms, total: 64.2 ms
Wall time: 29.5 s


In [12]:
df.head(10)

,year,temp_max,temp_min
0,1975,35.5,-35.4
1,1994,34.7,-36.6
2,2014,33.7,-34.6
3,1970,33.2,-39.5
4,1989,33.2,-37.2
5,1982,33.0,-42.2
6,1988,32.6,-31.5
7,1986,32.2,-38.4
8,2008,32.2,-32.7
9,2003,32.1,-37.2


**Task 1a:** What are the lowest and highest temperature measured each year for the period 1950 to 2014? Provide the list sorted in the descending order with respect to the maximum temperature. Extend the program to include the station number (not the station name) where the maximum/minimum temperature was measured.

In [21]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), year=int(p[1].split("-")[0]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014))

w = Window.partitionBy('year')

res_max = res.withColumn('temp_max', F.max('temp').over(w)).where(F.col('temp') == F.col('temp_max')).drop('temp')
res_min = res.withColumn('temp_min', F.min('temp').over(w)).where(F.col('temp') == F.col('temp_min')).drop('temp')

global df

df = res_max.join(res_min, "year").orderBy(['temp_max', 'year'], ascending=False).toPandas()

CPU times: user 75.2 ms, sys: 8.75 ms, total: 84 ms
Wall time: 37.2 s


In [22]:
df.head(10)

,year,station,temp_max,station,temp_min
0,1975,106100,35.5,122330,-35.4
1,1994,117160,34.7,113410,-36.6
2,2014,106160,33.7,124030,-34.6
3,1989,107400,33.2,114510,-37.2
4,1970,103080,33.2,114510,-39.5
5,1982,102540,33.0,113410,-42.2
6,1988,102540,32.6,114510,-31.5
7,2008,102390,32.2,112530,-32.7
8,1986,115230,32.2,112170,-38.4
9,2005,107140,32.1,112530,-39.1


## Assignment 2

Count the number of readings for each month in the period of 1950-2014 which are higher than 10 degrees. Repeat the exercise, this time taking only distinct readings from each station. That is, if a station reported a reading above 10 degrees in some month, then it appears only once in the count for that month. In this exercise you will use the `temperature-readings.csv` file.

The output should contain the following information:

Year, month, count

In [ ]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), date=p[1], year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), yearMonth=p[1][0:7], time=p[2], temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014))
res = res.filter(res['temp'] > 10.0)

res = res.groupBy('yearMonth').count().select('yearMonth', F.col('count'))
res = res.select(res['yearMonth'][0:4].cast("Int").alias('year'), res['yearMonth'][6:8].cast("Int").alias('month'), res['count']).orderBy(['year', 'month'], ascending=False)

global df
df = res.toPandas()

In [ ]:
df.head(10)

The following configuration is used for both SQL statement.

In [ ]:
db_temp = temperature_readings.map(lambda l: l.split(";")) \
            .filter(lambda m: (int(m[1][:4]) >= 1950 and int(m[1][:4]) <= 2014)) \
            .map(lambda m: Row(station = m[0], date = m[1], year = m[1].split("-")[0], month = m[1].split("-")[1],
                               time = m[2], temp = float(m[3]), quality = m[4]))

schema = sc_sql.createDataFrame(db_temp)

schema.registerTempTable("temp_schema")

In [ ]:
%%time

global res

res = sc_sql.sql("""
    SELECT year, month, COUNT(year) AS count
    FROM temp_schema
    WHERE year >= 1950 AND year <= 2014 and temp > 10
    GROUP BY year, month
    ORDER BY cast(year as integer) DESC, cast(month as integer) DESC
""")

res = res.toPandas()

In [ ]:
res.head(10)

Now we repeat the exercise where we take only distinct readings from each station.

In [ ]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), date=p[1], year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), yearMonth=p[1][0:7], time=p[2], temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014) & (res['temp'] > 10.0))

res = res.groupBy('year', 'month', 'station').count().groupBy('year', 'month').count().select('year', 'month', F.col('count')).orderBy(['year', 'month'], ascending=False)

global df
df = res.toPandas()

In [ ]:
df.head(10)

In [ ]:
%%time

global res

res = sc_sql.sql("""
    SELECT year, month, COUNT(year) AS count
    FROM (
        SELECT year, month, COUNT(year) AS count
        FROM temp_schema
        WHERE year >= 1950 AND year <= 2014 and temp > 10
        GROUP BY year, month, station
    )
    GROUP BY year, month
    ORDER BY cast(year as integer) DESC, cast(month as integer) DESC
""")

res = res.toPandas()

In [ ]:
res.head(10)

## Assignment 3

Find the average monthly temperature for each available station in Sweden. Your result should include average temperature for each station for each month in the period of 1960-2014. Bear in mind that not every station has the readings for each month in this timeframe. In this exercise you will use the `temperature-readings.csvfile`.

The output should contain the following information:

Year, month, station number, average monthly temperature

In [61]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), day=int(p[1].split("-")[2]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1960) & (res['year'] <= 2014))
res = res.groupBy('year', 'month', 'day', 'station')
res = res.agg(F.max('temp').alias('temp_max'), F.min('temp').alias('temp_min'))
res = res.withColumn('davg', (res['temp_max'] + res['temp_min'])/2)
res = res.select('year', 'month', 'day', 'station', 'davg')
res = res.groupBy('year', 'month', 'station').agg(F.mean('davg').alias('average-temperature'))

global df
df = res.orderBy(['year', 'month', 'station'], ascending=False).toPandas()

CPU times: user 235 ms, sys: 36.7 ms, total: 272 ms
Wall time: 40 s


In [62]:
df.head(10)

,year,month,station,average-temperature
0,2014,12,127130,-0.314516
1,2014,12,126440,-6.080645
2,2014,12,126290,-6.509677
3,2014,12,125440,-6.962903
4,2014,12,125170,-7.580645
5,2014,12,124300,-4.764516
6,2014,12,124110,-8.284375
7,2014,12,124030,-7.954839
8,2014,12,123460,-7.901613
9,2014,12,123340,-8.793548


## Assignment 4

Provide  a  list  of  stations  with  their  associated  maximum  measured  temperatures  and maximum  measured  daily  precipitation.  Show  only  those  stations  where  the  maximum temperature is between 25 and 30 degrees and maximum daily precipitation is between 100 mm and 200mm. In this exercise you will use the `temperature-readings.csv` and `precipitation-readings.csv` files. 

The output should contain the following information:

Station number, maximum measured temperature, maximum daily precipitation

In [41]:
%%time

res_t = temperature_readings.map(lambda l: l.split(";"))
res_p = precipitation_readings.map(lambda l: l.split(";"))

res_p = res_p.map(lambda p: Row(year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), day=int(p[1].split("-")[2]), station=int(p[0]), precipitation=float(p[3])))
res_p = sc_sql.createDataFrame(res_p)

res_p = res_p.groupBy('year', 'month', 'station', 'day')
res_p = res_p.agg(F.max('precipitation').alias('precip_max'))
res_p = res_p.groupBy('year', 'month', 'station')
res_p = res_p.agg(F.max('precip_max').alias('pmax'), F.min('precip_max').alias('pmin'))
res_p = res_p.filter((res_p['pmin'] >= 100.0) & (res_p['pmax'] <= 200.0))
res_p = res_p.groupBy('station')
res_p = res_p.agg(F.max('pmax').alias('max_daily_precipitation'))

res_t = res_t.map(lambda p: Row(station=int(p[0]), temperature=float(p[3])))
res_t = sc_sql.createDataFrame(res_t)

res_t = res_t.groupBy('station')
res_t = res_t.agg(F.max('temperature').alias('max_temperature'))
res_t = res_t.filter((res_t['max_temperature'] > 25.0) & (res_t['max_temperature'] < 30.0))

global df

df = res_t.join(res_p, "station").orderBy('station', ascending=False).toPandas()

CPU times: user 72 ms, sys: 42.1 ms, total: 114 ms
Wall time: 40 s


In [42]:
df.head()

,station,max_temperature,max_daily_precipitation
0,103100,31.3,6.8
1,102190,31.8,77.0
2,102170,29.1,3.2


## Assignment 5

Calculate the average monthly precipitation for the Östergotland region (list of stations is provided in the separate file) for the period 1993-2016. In order to do this, you will first need to calculate  the  total  monthly  precipitation  for  each  station  before calculating the monthly average (by averaging over stations). In this exercise you will use the `precipitation-readings.csv` and `stations-Ostergotland.csv` files.

HINT (not for the SparkSQL lab): Avoid using joins here! `stations-Ostergotland.csv` is small  and if distributed will cause a number of unnecessary shuffles when joined with precipitation RDD. If you distribute `precipitation-readings.csv` then either repartition your stations RDD to 1 partition or make use of the collect to acquire a python list and broadcast function to broadcast the list to all nodes.

The output should contain the following information:

Year, month, average monthly precipitation

In [ ]:
%%time

res_p = precipitation_readings.map(lambda l: l.split(";"))
res_s = station_oster_readings.map(lambda l: l.split(";"))

res_s = res_s.map(lambda m: int(m[0])).collect()
res_s = sc.broadcast(value=res_s)

res_p = res_p.map(lambda p: Row(station = int(p[0]), dateStation = p[1] + '-' + p[0], year = int(p[1][0:4]), month = int(p[1][8:10]), precipitation = float(p[3])))
res_p = sc_sql.createDataFrame(res_p)
res_p = res_p.filter((res_p['year'] >= 1993) & (res_p['year'] <= 2016))

res = res_p.where(res_p['station'].isin(res_s.value))

res = res.groupBy('dateStation').agg(F.sum('precipitation').alias('precipitation_cum'))
res = res.select(res['dateStation'][0:7].alias('yearMonth'), res['precipitation_cum'])
res = res.groupBy('yearMonth').agg(F.avg('precipitation_cum').alias('precipitation_avg'))
res = res.select(res['yearMonth'][0:4].alias('year'),
                 res['yearMonth'][6:8].alias('month'),
                 res['precipitation_avg'])

global df

df = res.orderBy(['year', 'month'], ascending=False).toPandas()

In [ ]:
df.head(10)

## Assignment 6

Compare the average monthly temperature (find the difference) in the period 1950-2014 for all stations in Östergotland with long-term monthly averages in the period of 1950-1980. Make a plot of your results. 

HINT: The first step is to find the monthly averages for each station. Then, you can average over all stations to acquire the average temperature for a specific year and month. This RDD/DataFrame can beu sed to compute the long-term average by averaging over all the years in the interval.

The out put should contain the following information:

Year, month, difference

**Answer:** It seems like this exercise has been removed from last year to this year, as the instructions have chnaged from 1. april to 2. april 2019.

In [ ]:
sc.stop()